En primer lugar cargamos e instalamos las librerías necesarias.

In [1]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

Error: must run as root
Leyendo lista de paquetes... Hecho
E: No se pudo abrir el fichero de bloqueo «/var/lib/apt/lists/lock» - open (13: Permission denied)
E: No se pudo bloquear el directorio /var/lib/apt/lists/
W: Se produjo un problema al desligar el fichero /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Se produjo un problema al desligar el fichero /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-l8o0968j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-l8o0968j
  Resolved https://github.com/huggingface/transformers to commit 3e39fd09a9c4d78bc2088777d3255789cf2acac8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━

In [1]:
from pytube import YouTube #Para descargar los videos
from pydub import AudioSegment
import pydub #Para trabajar con el audio

import os
import os.path as osp
from mlxtend.file_io import find_files
from mlxtend.utils import Counter

import re #Para la parte de normalizar
import string 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from autocorrect import Speller #Para los errores ortográficos
spell = Speller(lang='es')
from jiwer import wer #Para la métrica

[nltk_data] Downloading package punkt to /home/joheras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joheras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
device = 1 #gpu

Iniciamos sesión en Hugging Face.

In [36]:
from huggingface_hub import notebook_login

notebook_login()

Cargamos ahora los conjuntos de entrenamiento, validación y test con los que entrenaremos el modelo.

In [4]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "es", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "es", split="test", use_auth_token=True)

print(common_voice)

Found cached dataset common_voice_11_0 (/home/joheras/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/es/11.0.0/8975395f1d50a6b61f707acd3416761702d3b25412f5fb1004e1db51fe7c304a)
Found cached dataset common_voice_11_0 (/home/joheras/.cache/huggingface/datasets/mozilla-foundation___common_voice_11_0/es/11.0.0/8975395f1d50a6b61f707acd3416761702d3b25412f5fb1004e1db51fe7c304a)


DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 245987
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 15520
    })
})


Vemos que el dataset contiene más metadatos que simplemente las muestras de audio y su correspondiente texto transcrito, por lo que eliminamos esa información.

In [5]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])



Cargamos las características extraídas del modelo pre-entrenado. 

In [6]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")



Cargamos ahora el tokenizer, especificando que el idioma es el castellano.

In [7]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")

Podemos comprobar que funciona correctamente con el primer audio del dataset.

In [8]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)


Juntamos en una única clase el extractor de características y el tokenizador.

In [9]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Spanish", task="transcribe")


Mostramos el primer elemento del dataset para ver la forma que tienen los datos.

In [10]:
print(common_voice["train"][0])


{'audio': {'path': '/home/joheras/.cache/huggingface/datasets/downloads/extracted/ceff4655b28be26436dc18e6e7a158a9284dd27bfa22774fa0149e53fcd52aea/cv-corpus-11.0-2022-09-21/es/clips/common_voice_es_18306544.mp3', 'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -6.2212137e-05, -3.7340928e-05, -1.0736112e-06], dtype=float32), 'sampling_rate': 48000}, 'sentence': '¿ pero acaso tiene dispuesta su gente ?'}


Vemos que tenemos un vector de entrada de un audio y su correspondiente transcripción. 

Además, para poder pasar el extractor de características necesitamos que el sampling rate sea de 16kHz.

In [11]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


Vemos que se ha cambiado.

In [12]:
print(common_voice["train"][0])


{'audio': {'path': '/home/joheras/.cache/huggingface/datasets/downloads/extracted/ceff4655b28be26436dc18e6e7a158a9284dd27bfa22774fa0149e53fcd52aea/cv-corpus-11.0-2022-09-21/es/clips/common_voice_es_18306544.mp3', 'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -8.5134916e-06, -4.9502905e-05, -4.9611117e-05], dtype=float32), 'sampling_rate': 16000}, 'sentence': '¿ pero acaso tiene dispuesta su gente ?'}


Ahora escribimos una función que prepare los datos para el modelo.

In [13]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


In [14]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["test"])


  0%|          | 0/245987 [00:00<?, ?ex/s]

  0%|          | 0/15520 [00:00<?, ?ex/s]

Entrenamos ahora el modelo.

In [15]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [16]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


A continuación definimos la métrica con la que vamos a evaluar el modelo.

In [17]:
import evaluate

metric = evaluate.load("wer")


Ahora definimos una función que, dadas las predicciones del modelo, devuelve la métrica.

In [18]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


Cargamos el modelo de Whisper pre-entrenado.

In [19]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


Downloading:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967M [00:00<?, ?B/s]

In [20]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


In [21]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-es",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    group_by_length=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)


In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)


max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


Entrenamos el modelo.

In [25]:
import time
inicio = time.time()
trainer.train()
fin = time.time()

print((fin-inicio)/3600) #para ver cuántas horas ha tardado en entrenarse el modelo.


/home/joheras/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 245987
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4000
  Number of trainable parameters = 241734912
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
/home/joheras/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueez

Step,Training Loss,Validation Loss,Wer
1000,0.228100,0.238935,14.776187
2000,0.216400,0.218232,13.670787
3000,0.188200,0.205699,12.744958
4000,0.184200,0.199838,12.596617


`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...
`use_cache = True` is incompatible with gradient c

45.91475102292167


In [71]:
kwargs = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: es, split: test",
    "language": "es",
    "model_name": "Whisper Small Hi - Mirari",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}

In [77]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model = WhisperForConditionalGeneration.from_pretrained("mirari/whisper-small-es")
processor = WhisperProcessor.from_pretrained("mirari/whisper-small-es")

loading configuration file config.json from cache at /home/joheras/.cache/huggingface/hub/models--mirari--whisper-small-es/snapshots/e0c3f89d2a72df0730a85aa9f434f603b865a5a3/config.json
Model config WhisperConfig {
  "_name_or_path": "openai/whisper-small",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": null,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 448,
  "max_source_positions": 1500,
  "max_target_positions": 448,
  "model_type": "whisper",

OSError: Can't load tokenizer for 'mirari/whisper-small-es'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'mirari/whisper-small-es' is the correct path to a directory containing all relevant files for a WhisperTokenizer tokenizer.

In [75]:
from transformers import pipeline

pipe = pipeline(model="mirari/whisper-small-es")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text


loading configuration file config.json from cache at /home/joheras/.cache/huggingface/hub/models--mirari--whisper-small-es/snapshots/e0c3f89d2a72df0730a85aa9f434f603b865a5a3/config.json
Model config WhisperConfig {
  "_name_or_path": "mirari/whisper-small-es",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "WhisperForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "begin_suppress_tokens": [
    220,
    50257
  ],
  "bos_token_id": 50257,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 50258,
  "dropout": 0.0,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 50257,
  "forced_decoder_ids": null,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 448,
  "max_source_positions": 1500,
  "max_target_positions": 448,
  "model_type": "whispe

ValueError: This tokenizer cannot be instantiated. Please make sure you have `sentencepiece` installed in order to use this tokenizer.